# Block Sorting Algorithm
This optimization is modeled after the *multiple knapsacks problem*, in which the goal is to pack a set of items, with given values and sizes into a container with a maximum capacity. In this implementation, the goal is to assign blocks of leads to the Cadwell amplifiers (A-D, 9 rows each).

---
> Author:    Justin Campbell    
> Contact:   justin.campbell@hsc.utah.edu   
> Version:   04/10/2024  

In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

/Users/justincampbell/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/justincampbell/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


### Define Block Map and Amplifier Constraints

In [2]:
# Filepath for block map (if exists)
# block_map_file = '/Users/justincampbell/Desktop/BlockMap.csv'

In [3]:
# Amplifier constraints
amp_labels = ['A', 'B', 'C', 'D']
amp_size = 9
available_amps = 4 # modify if D is occupied by microelectrodes

In [4]:
# Load if block map file exists, otherwise specify manually
try:
    block_map = pd.read_csv(block_map_file)
    block_sizes = block_map['Size'].tolist()
    block_values = block_map['Value'].tolist()
    block_labels = block_map['Label'].tolist()

except:
    block_sizes = [
        5, 5, 4, 4, 
        5, 5, 5, 4, 
        4, 4, 5, 5, 
        5, 2, 2, 2]

    block_values = [ 
        1, 1, 1, 1, 
        1, 1, 1, 1, 
        1, 1, 1, 1, 
        1, 1, 1, 1]

    block_labels = [
        'LCM', 'LAINS', 'LPHIP', 'LAHIP', 
        'LPINS', 'RAINS', 'RPINS', 'RPHIP', 
        'RAHIP', 'RAMY', 'RACC', 'ROFC', 
        'RANT', 'bLACC', 'bLOFC', 'bLAMY']
    
    block_map = pd.DataFrame({'Label': block_labels, 'Size': block_sizes, 'Value': block_values})

# Display block map
print('%i Blocks\nSize = %i (Amp Capacity = %i)' % (len(block_map), sum(block_map['Size']), amp_size * available_amps))
block_map

16 Blocks
Size = 66 (Amp Capacity = 36)


,Label,Size,Value
0,LCM,5,1
1,LAINS,5,1
2,LPHIP,4,1
3,LAHIP,4,1
4,LPINS,5,1
5,RAINS,5,1
6,RPINS,5,1
7,RPHIP,4,1
8,RAHIP,4,1
9,RAMY,4,1


### Setup Solver

In [5]:
# Create variables
data = {}
data["sizes"] = block_sizes
data["values"] = block_values
data["num_blocks"] = len(data["sizes"])
data["all_blocks"] = range(data["num_blocks"])
data["amp_capacity"] = [amp_size] * available_amps
data["num_amps"] = len(data["amp_capacity"])
data["all_amps"] = range(data["num_amps"])

In [6]:
solver = pywraplp.Solver.CreateSolver("SCIP")

x = {}
for i in data["all_blocks"]:
    for b in data["all_amps"]:
        x[i, b] = solver.BoolVar(f"x_{i}_{b}")

# Each block is assigned to at most one amplifier
for i in data["all_blocks"]:
    solver.Add(sum(x[i, b] for b in data["all_amps"]) <= 1)

# The number of blocks cannot exceed the amplifier capacity
for b in data["all_amps"]:
    solver.Add(
        sum(x[i, b] * data["sizes"][i] for i in data["all_blocks"])
        <= data["amp_capacity"][b])
    
# Maximize total value of blocks in amplifiers
objective = solver.Objective()
for i in data["all_blocks"]:
    for b in data["all_amps"]:
        objective.SetCoefficient(x[i, b], data["values"][i])
        
objective.SetMaximization()

### Find Optimal Solution

In [7]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Suggested Order (Total Combined Value = %.1f):\n' %objective.Value())
    
    total_weight = 0
    for b in range(available_amps):
        print('Amp %s' %amp_labels[b])
        bin_weight = 0
        bin_value = 0
        for i in range(len(block_labels)):
            if x[i, b].solution_value() > 0:
                print('%s (Size: %i, Value: %.1f)' % (block_labels[i], data["sizes"][i], data["values"][i]))
                bin_weight += data["sizes"][i]
                bin_value += data["values"][i]
        print('Blocks Used: %i/%i\n' %(bin_weight, data["amp_capacity"][b]))
        total_weight += bin_weight
        
    # Display blocks that are not assigned in map
    print('\nUnused Blocks:\n')
    for i in range(len(block_labels)):
        used = False
        for b in range(available_amps):
            if x[i, b].solution_value() > 0:
                used = True
                break
        if not used:
            print('%s (Size: %i, Value: %.1f)' % (block_labels[i], data["sizes"][i], data["values"][i]))

else:
    print('The problem does not have an optimal solution.')

Suggested Order (Total Combined Value = 9.0):

Amp A
LCM (Size: 5, Value: 1.0)
bLOFC (Size: 2, Value: 1.0)
Blocks Used: 7/9

Amp B
LAHIP (Size: 4, Value: 1.0)
bLACC (Size: 2, Value: 1.0)
bLAMY (Size: 2, Value: 1.0)
Blocks Used: 8/9

Amp C
LPHIP (Size: 4, Value: 1.0)
RPHIP (Size: 4, Value: 1.0)
Blocks Used: 8/9

Amp D
RAHIP (Size: 4, Value: 1.0)
RAMY (Size: 4, Value: 1.0)
Blocks Used: 8/9


Unused Blocks:

LAINS (Size: 5, Value: 1.0)
LPINS (Size: 5, Value: 1.0)
RAINS (Size: 5, Value: 1.0)
RPINS (Size: 5, Value: 1.0)
RACC (Size: 5, Value: 1.0)
ROFC (Size: 5, Value: 1.0)
RANT (Size: 5, Value: 1.0)
